En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [60]:
%reload_ext autoreload
%autoreload 2

print("IPython autoreload activated")

IPython autoreload activated


In [21]:
import os

from google.colab import auth
auth.authenticate_user()


In [45]:
from google.cloud import bigquery

# Crear una instancia del cliente
client = bigquery.Client("tw-techdash")

In [64]:
!pip install memory-profiler



In [25]:
# Local file paths (consider user input/environment variables for flexibility)
MOUNT_POINT = os.environ['LC_MOUNT_POINT'] = "/content/drive"
SOURCE_PATH = os.environ['LC_SOURCE_PATH'] = "/content/drive/Othercomputers/My MacBook Pro/challenge_DE"


In [26]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
# Mount Google Drive (optional, depending on usage context)
# Uncomment the following line if mounting is needed:
drive.mount(MOUNT_POINT, force_remount=True)

# Change directory using a more explicit method
target_directory = os.path.join(MOUNT_POINT, SOURCE_PATH, "src")
os.chdir(target_directory)

# Confirm the current directory:
print(f"Changed directory to: {os.getcwd()}")  # Use os.getcwd() for accuracy

Mounted at /content/drive
Changed directory to: /content/drive/Othercomputers/My MacBook Pro/challenge_DE/src


In [75]:
query = """
WITH datos AS (
  SELECT
    FORMAT_TIMESTAMP('%Y-%m-%d', TIMESTAMP(date)) AS fecha_formateada,
    user.username as usuario
  FROM
    `tw-techdash.de_challenge_latam.tweets_latam`
),

conteo_por_fecha AS (
  SELECT
    fecha_formateada,
    COUNT(usuario) AS num_tweets
  FROM
    datos
  GROUP BY
    fecha_formateada
  ORDER BY
    num_tweets DESC
  LIMIT 10
),

usuario_top_por_fecha AS (
  SELECT
    fecha_formateada,
    usuario,
    COUNT(usuario) AS num_tweets_por_usuario,
    RANK() OVER (PARTITION BY fecha_formateada ORDER BY COUNT(usuario) DESC) AS rank
  FROM
    datos
  WHERE
    fecha_formateada IN (SELECT fecha_formateada FROM conteo_por_fecha)
  GROUP BY
    fecha_formateada, usuario
)

SELECT
  u.usuario AS top_user,
  u.fecha_formateada as date,
FROM
  usuario_top_por_fecha u
JOIN
  conteo_por_fecha c
ON
  u.fecha_formateada = c.fecha_formateada
WHERE
  u.rank = 1
ORDER BY
  c.num_tweets DESC;

"""

query_job = client.query(query)

query_job.result()

results = query_job.result()

output = [(row.top_user, row.date) for row in results]

print("QUERY 1")
print ("Las top 10 fechas donde hay más tweets. Ordenadas por usuario (username) que más publicaciones tiene por cada uno de esos días: ")
print(output)

QUERY 1
Las top 10 fechas donde hay más tweets. Ordenadas por usuario (username) que más publicaciones tiene por cada uno de esos días: 
[('RanbirS00614606', '2021-02-12'), ('MaanDee08215437', '2021-02-13'), ('RaaJVinderkaur', '2021-02-17'), ('jot__b', '2021-02-16'), ('rebelpacifist', '2021-02-14'), ('neetuanjle_nitu', '2021-02-18'), ('jot__b', '2021-02-15'), ('MangalJ23056160', '2021-02-20'), ('Surrypuria', '2021-02-23'), ('Preetm91', '2021-02-19')]


In [76]:
query1= """
  WITH datos AS (
    SELECT
      FORMAT_TIMESTAMP('%Y-%m-%d', TIMESTAMP(date)) AS fecha_formateada,
      user.username as usuario
    FROM
      `tw-techdash.de_challenge_latam.tweets_latam`
  ),

  conteo_por_fecha AS (
    SELECT
      fecha_formateada,
      COUNT(usuario) AS num_tweets
    FROM
      datos
    GROUP BY
      fecha_formateada
    ORDER BY
      num_tweets DESC
    LIMIT 10
  ),

  usuario_top_por_fecha AS (
    SELECT
      fecha_formateada,
      usuario,
      COUNT(usuario) AS num_tweets_por_usuario,
      RANK() OVER (PARTITION BY fecha_formateada ORDER BY COUNT(usuario) DESC) AS rank
    FROM
      datos
    WHERE
      fecha_formateada IN (SELECT fecha_formateada FROM conteo_por_fecha)
    GROUP BY
      fecha_formateada, usuario
  )

  SELECT
    u.usuario AS top_user,
    u.fecha_formateada as date,
  FROM
    usuario_top_por_fecha u
  JOIN
    conteo_por_fecha c
  ON
    u.fecha_formateada = c.fecha_formateada
  WHERE
    u.rank = 1
  ORDER BY
    c.num_tweets DESC
"""

In [67]:
%run q1_memory.py
q1_memory

<function __main__.q1_memory(client: google.cloud.bigquery.client.Client, query: str) -> Tuple[List[Tuple[str, str]], float]>

In [73]:
q1_memory(client, query1)

         22238 function calls (22154 primitive calls) in 1.933 seconds

   Ordered by: cumulative time
   List reduced from 671 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      5/4    0.000    0.000    1.928    0.482 retry_unary.py:286(retry_wrapped_func)
      5/4    0.000    0.000    1.928    0.482 retry_unary.py:85(retry_target)
        3    0.000    0.000    1.927    0.642 client.py:814(_call_api)
        3    0.000    0.000    1.926    0.642 __init__.py:383(api_request)
        3    0.000    0.000    1.925    0.642 __init__.py:276(_make_request)
        3    0.000    0.000    1.925    0.642 __init__.py:345(_do_request)
        3    0.000    0.000    1.924    0.641 requests.py:482(request)
        5    0.000    0.000    1.921    0.384 sessions.py:500(request)
        5    0.000    0.000    1.896    0.379 sessions.py:673(send)
        5    0.001    0.000    1.889    0.378 adapters.py:613(send)


Tiempo total de ejecución: 1

([('RanbirS00614606', '2021-02-12'),
  ('MaanDee08215437', '2021-02-13'),
  ('RaaJVinderkaur', '2021-02-17'),
  ('jot__b', '2021-02-16'),
  ('rebelpacifist', '2021-02-14'),
  ('neetuanjle_nitu', '2021-02-18'),
  ('jot__b', '2021-02-15'),
  ('MangalJ23056160', '2021-02-20'),
  ('Surrypuria', '2021-02-23'),
  ('Preetm91', '2021-02-19')],
 1.9340806007385254)

In [65]:
%run q1_time.py
q1_time

<function __main__.q1_time(client: google.cloud.bigquery.client.Client, query: str) -> Tuple[List[Tuple[str, str]], float]>

In [71]:
q1_time(client, query1)

         14647 function calls (14575 primitive calls) in 1.270 seconds

   Ordered by: cumulative time
   List reduced from 580 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      5/4    0.000    0.000    1.268    0.317 retry_unary.py:286(retry_wrapped_func)
      5/4    0.000    0.000    1.268    0.317 retry_unary.py:85(retry_target)
        3    0.000    0.000    1.268    0.423 client.py:814(_call_api)
        3    0.000    0.000    1.268    0.423 __init__.py:383(api_request)
        3    0.000    0.000    1.267    0.422 __init__.py:276(_make_request)
        3    0.000    0.000    1.267    0.422 __init__.py:345(_do_request)
        3    0.000    0.000    1.267    0.422 requests.py:482(request)
        3    0.000    0.000    1.267    0.422 sessions.py:500(request)
        3    0.000    0.000    1.263    0.421 sessions.py:673(send)
        3    0.000    0.000    1.261    0.420 adapters.py:613(send)


Tiempo total de ejecución: 1

([('RanbirS00614606', '2021-02-12'),
  ('MaanDee08215437', '2021-02-13'),
  ('RaaJVinderkaur', '2021-02-17'),
  ('jot__b', '2021-02-16'),
  ('rebelpacifist', '2021-02-14'),
  ('neetuanjle_nitu', '2021-02-18'),
  ('jot__b', '2021-02-15'),
  ('MangalJ23056160', '2021-02-20'),
  ('Surrypuria', '2021-02-23'),
  ('Preetm91', '2021-02-19')],
 1.2697055339813232)

In [47]:
import time

query = """
WITH datos AS (
  SELECT
    FORMAT_TIMESTAMP('%Y-%m-%d', TIMESTAMP(date)) AS fecha_formateada,
    user.username as usuario
  FROM
    `tw-techdash.de_challenge_latam.tweets_latam`
),

conteo_por_fecha AS (
  SELECT
    fecha_formateada,
    COUNT(usuario) AS num_tweets
  FROM
    datos
  GROUP BY
    fecha_formateada
  ORDER BY
    num_tweets DESC
  LIMIT 10
),

usuario_top_por_fecha AS (
  SELECT
    fecha_formateada,
    usuario,
    COUNT(usuario) AS num_tweets_por_usuario,
    RANK() OVER (PARTITION BY fecha_formateada ORDER BY COUNT(usuario) DESC) AS rank
  FROM
    datos
  WHERE
    fecha_formateada IN (SELECT fecha_formateada FROM conteo_por_fecha)
  GROUP BY
    fecha_formateada, usuario
)

SELECT
  u.usuario AS top_user,
  u.fecha_formateada as date,
FROM
  usuario_top_por_fecha u
JOIN
  conteo_por_fecha c
ON
  u.fecha_formateada = c.fecha_formateada
WHERE
  u.rank = 1
ORDER BY
  c.num_tweets DESC;

"""

start_time = time.time()

query_job = client.query(query)

query_job.result()

results = query_job.result()

output = [(row.top_user, row.date) for row in results]

end_time = time.time()
execution_time = end_time - start_time

print("QUERY 1")
print ("Las top 10 fechas donde hay más tweets. Ordenadas por usuario (username) que más publicaciones tiene por cada uno de esos días: ")
print(output)
print(f"Tiempo total de ejecución: {execution_time:.2f} segundos")

QUERY 1
Las top 10 fechas donde hay más tweets. Ordenadas por usuario (username) que más publicaciones tiene por cada uno de esos días: 
[('RanbirS00614606', '2021-02-12'), ('MaanDee08215437', '2021-02-13'), ('RaaJVinderkaur', '2021-02-17'), ('jot__b', '2021-02-16'), ('rebelpacifist', '2021-02-14'), ('neetuanjle_nitu', '2021-02-18'), ('jot__b', '2021-02-15'), ('MangalJ23056160', '2021-02-20'), ('Surrypuria', '2021-02-23'), ('Preetm91', '2021-02-19')]
Tiempo total de ejecución: 1.09 segundos


In [46]:
import cProfile
import pstats

import time

# Definir la consulta SQL
query = """
WITH datos AS (
  SELECT
    FORMAT_TIMESTAMP('%Y-%m-%d', TIMESTAMP(date)) AS fecha_formateada,
    user.username as usuario
  FROM
    `tw-techdash.de_challenge_latam.tweets_latam`
),

conteo_por_fecha AS (
  SELECT
    fecha_formateada,
    COUNT(usuario) AS num_tweets
  FROM
    datos
  GROUP BY
    fecha_formateada
  ORDER BY
    num_tweets DESC
  LIMIT 10
),

usuario_top_por_fecha AS (
  SELECT
    fecha_formateada,
    usuario,
    COUNT(usuario) AS num_tweets_por_usuario,
    RANK() OVER (PARTITION BY fecha_formateada ORDER BY COUNT(usuario) DESC) AS rank
  FROM
    datos
  WHERE
    fecha_formateada IN (SELECT fecha_formateada FROM conteo_por_fecha)
  GROUP BY
    fecha_formateada, usuario
)

SELECT
  u.usuario AS top_user,
  u.fecha_formateada as date
FROM
  usuario_top_por_fecha u
JOIN
  conteo_por_fecha c
ON
  u.fecha_formateada = c.fecha_formateada
WHERE
  u.rank = 1
ORDER BY
  c.num_tweets DESC;
"""

def run_query():
    query_job = client.query(query)
    query_job.result()
    results = query_job.result()
    output = [(row.top_user, row.date) for row in results]

    print("QUERY 1")
    print("Las top 10 fechas donde hay más tweets. Ordenadas por usuario (username) que más publicaciones tiene por cada uno de esos días:")
    print(output)

# Medir el tiempo total de ejecución y usar cProfile para obtener detalles
profiler = cProfile.Profile()
profiler.enable()

start_time = time.time()

# Ejecutar la consulta
run_query()

end_time = time.time()
execution_time = end_time - start_time

profiler.disable()

# Mostrar las estadísticas del profiler
stats = pstats.Stats(profiler)
stats.strip_dirs().sort_stats('cumtime').print_stats(10)  # Mostrar las 10 líneas más lentas

print(f"Tiempo total de ejecución: {execution_time:.2f} segundos")


QUERY 1
Las top 10 fechas donde hay más tweets. Ordenadas por usuario (username) que más publicaciones tiene por cada uno de esos días:
[('RanbirS00614606', '2021-02-12'), ('MaanDee08215437', '2021-02-13'), ('RaaJVinderkaur', '2021-02-17'), ('jot__b', '2021-02-16'), ('rebelpacifist', '2021-02-14'), ('neetuanjle_nitu', '2021-02-18'), ('jot__b', '2021-02-15'), ('MangalJ23056160', '2021-02-20'), ('Surrypuria', '2021-02-23'), ('Preetm91', '2021-02-19')]
         27471 function calls (27371 primitive calls) in 3.042 seconds

   Ordered by: cumulative time
   List reduced from 739 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000    3.042    0.608 interactiveshell.py:3512(run_code)
        5    0.000    0.000    3.041    0.608 {built-in method builtins.exec}
        1    0.000    0.000    3.041    3.041 <ipython-input-46-6cea65cd9e77>:1(<cell line: 75>)
        1    0.000    0.000    3.041    3.041 <ipython-inpu